In [1]:
import chess
from chess.pgn import read_game
import numpy as np
from anytree import Node
from network import ChessNeuralNetwork

In [2]:
def board_generator():
    pgn = open("millionbase-2.22.pgn")
    while True:
        game = read_game(pgn)
        if game and len(list(game.main_line()))>0:
            node = game
            move_number = np.random.randint(0, high=len(list(game.main_line())) - 1) #don't take the last move
            while 2*(node.board().fullmove_number-1)+int(not node.board().turn) < move_number:
                next_node = node.variation(0)
                node = next_node
            yield node.board()
        else:
            pgn.seek(0)

In [3]:
def simple_value(board):
    values = [1, 3, 3, 5, 9]
    s = 0
    for i, v in enumerate(values):
        s += ChessNeuralNetwork.pad_bitmask(board.pieces_mask(i + 1, 1)).sum() * v
        s -= ChessNeuralNetwork.pad_bitmask(board.pieces_mask(i + 1, 0)).sum() * v
    return np.tanh(s)

In [4]:
def alpha_beta(node, depth, alpha, beta):
    if depth == 0 or node.board.is_game_over():
        return simple_value(node.board), node

    legal_moves = list(node.board.legal_moves)
    child_boards = [node.board.copy() for _ in legal_moves]
    children = []
    for idx in range(len(node.board.legal_moves)):
        child_boards[idx].push(legal_moves[idx])
        child = Node(str(legal_moves[idx]), parent=node, board=child_boards[idx], move=legal_moves[idx])
        children.append(child)
    n = node
    if node.board.turn:
        v = -100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv > v:
                v = vv
                n = nn
            alpha = max(alpha, v)
            if beta <= alpha:
                break  # (* β cut-off *)
        return v, n
    else:
        v = 100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv < v:
                v = vv
                n = nn
            beta = min(beta, v)
            if beta <= alpha:
                break  # (* α cut-off *)
        return v, n

In [5]:
gen = board_generator()
b = gen.__next__()
root = Node('root', board=b, move=None)
v, n = alpha_beta(root, 2, -100000, 100000)
print(v, n)

0.0 Node('/root/d5c4/f4c7', board=Board('rnbqkb1r/ppB1pp1p/5np1/8/2pP4/2N5/PP2PPPP/R2QKBNR b KQkq - 0 5'), move=Move.from_uci('f4c7'))


In [6]:
def make_feature_vector(board):
    piece_matrix = np.zeros((64, len(chess.PIECE_TYPES) + 1, len(chess.COLORS)))

    # piece positions
    for piece in chess.PIECE_TYPES:
        for color in chess.COLORS:
            piece_matrix[:, piece, int(color)] = ChessNeuralNetwork.pad_bitmask(board.pieces_mask(piece, color))

    # en passant target squares
    if board.ep_square:
        piece_matrix[board.ep_square, -1, int(board.turn)] = 1

    reshaped_piece_matrix = piece_matrix.reshape((64, (len(chess.PIECE_TYPES) + 1) * len(chess.COLORS)))
    feature_array = np.zeros((64, (len(chess.PIECE_TYPES) + 1) * len(chess.COLORS) + 2))
    feature_array[:, :-2] = reshaped_piece_matrix

    # empty squares
    empty_squares = (reshaped_piece_matrix.sum(axis=1) == 0)
    feature_array[empty_squares, :-2] = 1

    # castling rights
    feature_array[:, -1] = ChessNeuralNetwork.pad_bitmask(board.castling_rights)

    feature_vector = np.zeros((1, 1025))
    feature_vector[0, :-1] = np.reshape(feature_array, (1024,))
    feature_vector[0, -1] = board.turn

    return feature_vector

In [7]:
W_1 = np.zeros((1025, 100))
pieces = ['p', 'n', 'b', 'r', 'q', 'P', 'N', 'B', 'R', 'Q']
values = [-1, -3, -3, -5, -9, 1, 3, 3, 5, 9]
for piece, value in zip(pieces, values):
    fen = '/'.join([8*piece for _ in range(8)]) + ' b -- - 0 1'
    board = chess.Board(fen)
    W_1[ChessNeuralNetwork.make_feature_vector(board)[0] == 1, 0] = value

In [7]:
import tensorflow as tf
sess = tf.Session()
network = ChessNeuralNetwork()
sess.run(tf.global_variables_initializer())

In [16]:
gen = board_generator()
board = gen.__next__()
feature_vector = ChessNeuralNetwork.make_feature_vector(board)
print(sess.run(network.value, feed_dict={network.feature_vector_: feature_vector})[0,0])
print(simple_value(board))

0.0
0.0
